In [114]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import codecs
import lxml
import os

### Testing one file

In [96]:
# import
file = codecs.open("anime_6501.html", "r", "utf-8")

In [97]:
# beautify using BS4
soup = BeautifulSoup(file, 'html.parser')

In [98]:
#getting the title of the anime from the tag "strong"
soup.find_all('strong')[0].contents[0]

'Tactical Roar'

## For the bunch of html files

In [286]:
animeTitle = []
animeTypes = []
animeNumEpisode = []
releaseDate = []
endDate = []
animeNumMembers = []
animeScore = []
animeUsers = []
animeRank = []
animePopularity = []
animeDescription = []
animeRelated = []
animeCharacters = []
animeVoices = []
animeStaff = []
for i in range(5):
    # anime titles
    path = "anime_650" + str(i+1) + ".html"
    file = codecs.open(path, "r", "utf-8")
    soup = BeautifulSoup(file, 'html.parser')
    animeTitle.append(soup.find_all('strong')[0].contents[0])
    
    # left sied of the html------------------------------------------------------------
    divs = soup.find_all("div", {"class": "spaceit_pad"})
    for div in divs:
        spans = div.find_all("span")
        for span in spans:
            # anime types
            if span.contents[0] == 'Type:':
                animeTypes.append(div.find_all('a')[0].contents[0])
            # anime number of episodes
            if span.contents[0] == 'Episodes:':
                animeNumEpisode.append(int(div.contents[2]))
            # anime release and end dates
            if span.contents[0] == 'Aired:':
                if len(div.contents[2]) > 21:
                    releaseDate.append(pd.to_datetime(div.contents[2][4:16]))
                    endDate.append(pd.to_datetime(div.contents[2][19:-4]))
                else:
                    releaseDate.append(pd.to_datetime(div.contents[2][4:-4]))
                    endDate.append('-')
                    
    # middle side of the html------------------------------------------------------------
    divs = soup.find_all("div", {"class": "stats-block po-r clearfix"})
    for div in divs:
        # anime number of members
        members = div.find_all("span", {"class": "numbers members"})
        animeNumMembers.append(int(members[0].contents[1].contents[0].replace(',', '')))
        # anime score
        score = div.find_all("div", {"class": "score-label score-6"})
        animeScore.append(float(score[0].contents[0]))
        # anime number of users
        users = div.find_all("div", {"class": "fl-l score"})
        animeUsers.append(int(users[0]['data-user'][:-6].replace(',', '')))
        # anime rank
        rank = div.find_all("span", {"class": "numbers ranked"})
        animeRank.append(int(rank[0].contents[1].contents[0][1:]))
        # anime popularity
        popularity = div.find_all("span", {"class": "numbers popularity"})
        animePopularity.append(int(popularity[0].contents[1].contents[0][1:]))
        
    # anime description
    description = soup.find_all("p", {"itemprop": "description"})
    for br in description[0].find_all("br"):
        br.replace_with("\n")
    animeDescription.append(''.join(description[0].contents))
    
    # related animes 
    related = soup.find_all("table", {"class": "anime_detail_related_anime"})
    x = []
    y = []
    for tr in related:
        td = tr.find_all("td")
        for i in range(0, len(td), 2):
            x.append(td[i].contents[0])
            t = td[i+1].find_all("a")
            y.append(t[0].contents[0])
        animeRelated.append('\n'.join([f'{x} {y}' for x, y in dict(zip(x, y)).items()]).split('\n'))
        
    # anime characters and voices
    characters = soup.find_all("div", {"class": "detail-characters-list clearfix"})
    chars = characters[0].find_all("h3", {"class": "h3_characters_voice_actors"})
    x = []
    y = []
    voices = characters[0].find_all("td", {"class": "va-t ar pl4 pr4"})
    for i in chars:
        x.append(i.contents[0].contents[0])
    for i in voices:
        y.append(i.contents[1].contents[0])
    animeCharacters.append(x)
    animeVoices.append(y)
    
    # anime staff
    staff = soup.find_all("div", {"class": "detail-characters-list clearfix"})
    staff = staff[1].find_all("td")
    x = []
    y = []
    for i in range(1, len(staff), 2):
        x.append(staff[i].contents[1].contents[0])
        y.append(staff[i].find_all("small")[0].contents[0])
    animeStaff.append([list(i) for i in list(zip(x,y))])

In [288]:
df = pd.DataFrame(
    [animeTitle, animeTypes, animeNumEpisode, releaseDate, endDate, animeNumMembers, animeScore, animeUsers, animeRank, animePopularity, animeDescription, animeRelated, animeCharacters, animeVoices, animeStaff], 
    index=['Title', 'Type', 'Episodes','Release date', 'End date', 'Members', 'Score', 'Users', 'Rank', 'Popularity', 'Description', 'Related', 'Characters', 'Voices', 'Staff']
).T
df

,Title,Type,Episodes,Release date,End date,Members,Score,Users,Rank,Popularity,Description,Related,Characters,Voices,Staff
0,Tactical Roar,TV,13,2006-01-08,2006-04-02 00:00:00,9314,6.43,3322,6508,5451,In the near future the world's climate shifted...,[Side story: Tactical Roar Special],"[Misaki, Nanaha, Watatsumi, Tsubasa, Nagimiya,...","[Nakahara, Mai, Takahashi, Mikako, Suganuma, H...","[[Aketagawa, Jin, Sound Director], [Ishikura, ..."
1,Takamiya Nasuno Desu!: Teekyuu Spin-off,TV,12,2015-04-07,2015-06-23 00:00:00,13063,6.43,6590,6509,4753,A spin-off from Teekyuu which centers on the d...,"[Adaptation: Takamiya Nasuno desu!, Parent sto...","[Takamiya, Nasuno, Oshimoto, Youta, Bandou, Ma...","[Narumi, Kyoko, Oosaka, Ryouta, Hanazawa, Kana...","[[Shiraishi, Naoko, Producer], [Itagaki, Shin,..."
2,Takunomi.,TV,12,2018-01-12,2018-03-30 00:00:00,40770,6.43,12145,6510,2783,20-year-old Michiru Amatsuki moved to Tokyo af...,[Adaptation: Takunomi.],"[Kiriyama, Nao, Amatsuki, Michiru, Kiriyama, M...","[Anzai, Chika, Imamura, Ayaka, Uchida, Maaya, ...","[[Kobayashi, Tomoki, Director], [Hamano, Takat..."
3,Transformers: The☆Headmasters,TV,35,1987-07-03,1988-03-28 00:00:00,3422,6.43,1462,6511,8045,Headmasters starts with Galvatron leading a ne...,"[Sequel: Transformers: Choujin Master Force, A...","[Prime, Optimus, Galvatron, Prime, Rodimus, Fo...","[Genda, Tesshou, Katou, Seizou, Ishimaru, Hiro...","[[Sasaki, Katsutoshi, Director], [Kageyama, Hi..."
4,UFO Robo Grendizer tai Great Mazinger,Movie,1,1976-03-20,-,1969,6.43,847,6512,9489,Alien invaders take over Great Mazinger who is...,"[Adaptation: UFO Robo Grendizer, Other: UFO Ro...","[Kabuto, Kouji, Fleed, Duke, Tsurugi, Tetsuya,...","[Ishimaru, Hiroya, Tomiyama, Kei, Noda, Keiich...","[[Kasai, Osamu, Episode Director], [Fujikawa, ..."


In [289]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Title         5 non-null      object        
 1   Type          5 non-null      object        
 2   Episodes      5 non-null      object        
 3   Release date  5 non-null      datetime64[ns]
 4   End date      5 non-null      object        
 5   Members       5 non-null      object        
 6   Score         5 non-null      object        
 7   Users         5 non-null      object        
 8   Rank          5 non-null      object        
 9   Popularity    5 non-null      object        
 10  Description   5 non-null      object        
 11  Related       5 non-null      object        
 12  Characters    5 non-null      object        
 13  Voices        5 non-null      object        
 14  Staff         5 non-null      object        
dtypes: datetime64[ns](1), object(14)
memory usag